In [1]:
import pandas as pd
import os

In [2]:
filefolder = r'C:\Users\johan\OneDrive\Marcobre\1.0 PROJECTS\01. Crear budget de mantenimiento\hr2csv'
filename = r'5._Carga de hoja de ruta - GRUPO 5.xlsx'

In [3]:
filerute = os.path.join(filefolder,filename)

In [10]:
db = pd.read_excel(filerute,sheet_name=0, skiprows = 6, header = None)

In [151]:
# Hojas de ruta
HR = db.loc[db.iloc[:,0] == 1,:]
HR.drop(labels=[11,12,13,14,15,16,17], axis = 1, inplace = True)
HRcolumnsname = [0,'Eq','Contador','HRDescripcion','HRPuesto','Centro','Utilizacion','GPlanificacion', 'EstadoGlobal','EstadoInstalacion','Estrategia']
HR.columns = HRcolumnsname
HR.loc[:,'HRContador'] = HR['Eq'] + HR['Contador']

In [43]:
EQlist = HR.drop_duplicates(subset = ['Eq'])
#EQlist.drop(labels = [0,2,3,4,5,6,7,8,9,10], axis=1, inplace= True)
EQlist.reset_index(drop= True,inplace=True)

In [166]:
# Posiciones de las Hojas de Ruta
PHR = db.loc[db.iloc[:,0] == 2,:]
PHR.drop(columns = [10,11,12,13,14,15,16,17],inplace = True)
PHRcolumnsname = [0,'Eq','pos','Puesto','ClaseOT','Actividades','HH','H','ACant','AHoras']
PHR.columns = PHRcolumnsname

In [45]:
# Actividad detallada de laPosiciones de las Hojas de Ruta
AHR = db.loc[db.iloc[:,0] == 3,:]

In [198]:
nm1=0
#HR.tail(1).index.values[0] = last index of HR dataframe
for m in PHR.index:
    for n in HR.index:
        if nm1<=m and m<=n:
            PHR.loc[m,'HRContador'] = HR.loc[nm1,'HRContador']
            break
        elif m>=HR.tail(1).index.values[0]:
            PHR.loc[m,'HRContador'] = HR.loc[HR.tail(1).index.values[0],'HRContador']
            break
        nm1 = n
PHR.loc[:,'posHRContador'] = PHR.loc[:,'HRContador'] + PHR.loc[:,'pos'].astype(str)

In [200]:
PHR.duplicated(subset = 'posHRContador').sum()

0

In [206]:
# Materiales de Hojas de Ruta
MHR = db.loc[db.iloc[:,0] == 4,:]
MHR = MHR.drop(columns = [4,5,6,7,8,9,10,11,12,13,14,15,16,17])
MHRcolumnsname = [0,'Eq','Material','MCantidad']
MHR.columns = MHRcolumnsname

In [214]:
nm1=0
#HR.tail(1).index.values[0] = last index of HR dataframe
for m in MHR.index:
    for n in PHR.index:
        if nm1<=m and m<=n:
            MHR.loc[m,'posHRContador'] = PHR.loc[nm1,'posHRContador']
            MHR.loc[m,'HRContador'] = PHR.loc[nm1,'HRContador']
            break
        elif m>=PHR.tail(1).index.values[0]:
            MHR.loc[m,'posHRContador'] = PHR.loc[HR.tail(1).index.values[0],'posHRContador']
            MHR.loc[m,'HRContador'] = PHR.loc[HR.tail(1).index.values[0],'HRContador']
            break
        nm1 = n
#PHR.loc[:,'posHRContador'] = PHR.loc[:,'HRContador'] + PHR.loc[:,'pos'].astype(str)

In [247]:
HR.head()

,0,Eq,Contador,HRDescripcion,HRPuesto,Centro,Utilizacion,GPlanificacion,EstadoGlobal,EstadoInstalacion,Estrategia,HRContador
0,1,MBOM020,M1,INSP_MEC_BOMBA_SUMMIT 2196MTO-7,TEC-MEC,JP11,4.0,P01,4.0,1.0,HOR002,MBOM020M1
24,1,MBOM020,M2,PREV_MEC_BOMBA_SUMMIT 2196MTO-7,TEC-MEC,JP11,4.0,P01,4.0,1.0,HOR002,MBOM020M2
67,1,MBOM020,E1,INSP_ELE_BOMBA_SUMMIT 2196MTO-7,TEC-ELE,JP11,4.0,P01,4.0,1.0,CAL001,MBOM020E1
77,1,MBOM020,E2,PREV_ELE_BOMBA_SUMMIT 2196MTO-7,TEC-ELE,JP11,4.0,P01,4.0,1.0,HOR002,MBOM020E2
143,1,MBOM020,I1,INSP_INS_BOMBA_SUMMIT 2196MTO-7,TEC-INS,JP11,4.0,P01,4.0,1.0,CAL001,MBOM020I1


In [245]:
allDutiesMat = pd.merge(PHR.loc[:,['Eq','pos','Actividades','Puesto','ClaseOT','ACant','AHoras','HRContador','posHRContador']],MHR.loc[:,['posHRContador','Material','MCantidad']],on='posHRContador',how='left')

In [246]:
allDutiesMat.head(13)

,Eq,pos,Actividades,Puesto,ClaseOT,ACant,AHoras,HRContador,posHRContador,Material,MCantidad
0,MBOM020,10,Inspeccion mantenimiento de acople 2000H,TEC-MEC,PM01,1.0,1.0,MBOM020M1,MBOM020M110,NaN,NaN
1,MBOM020,20,Inspeccion general bomba 330H,TEC-MEC,PM01,2.0,0.4,MBOM020M1,MBOM020M120,NaN,NaN
2,MBOM020,30,Inspeccion general bomba 660H,TEC-MEC,PM01,2.0,0.6,MBOM020M1,MBOM020M130,NaN,NaN
3,MBOM020,10,Cambio de impulsor 4000H,TEC-MEC,PM01,3.0,6.0,MBOM020M2,MBOM020M210,NaN,NaN
4,MBOM020,20,Cambio de sello mecanico 8000H,TEC-MEC,PM01,2.0,3.0,MBOM020M2,MBOM020M220,NaN,NaN
5,MBOM020,30,Cambio de eje 8000H,TEC-MEC,PM01,2.0,5.0,MBOM020M2,MBOM020M230,NaN,NaN
6,MBOM020,40,Cambio de elementos de union 8000H,TEC-MEC,PM01,2.0,1.0,MBOM020M2,MBOM020M240,NaN,NaN
7,MBOM020,50,Mtto general carcasa 8000H,TEC-MEC,PM01,2.0,18.0,MBOM020M2,MBOM020M250,NaN,NaN
8,MBOM020,10,Inspeccion general del motor 2S,TEC-ELE,PM01,2.0,0.5,MBOM020E1,MBOM020E110,NaN,NaN
9,MBOM020,10,Mantenimiento menor de motor 2000H,TEC-ELE,PM01,2.0,2.0,MBOM020E2,MBOM020E210,NaN,NaN
